<a href="https://colab.research.google.com/github/ahatasham5/Breast_Cancer_Prognosis/blob/main/test_alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <div style="color:yellow;display:inline-block;border-radius:5px;background-color:#007BA7;font-family:Nexa;overflow:hidden"><p style="padding:15px;color:yellow;overflow:hidden;font-size:100%;letter-spacing:0.5px;margin:0"><b> </b> Import Modules</p></div>

In [1]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

In [2]:
import torch
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
import torchvision.models as models

def create_model(model_name, num_classes):
    """Creates a model based on the given name.

    Args:
        model_name (str): Name of the model (e.g., 'resnet50', 'vgg16', 'alexnet')
        num_classes (int): Number of output classes for the model

    Returns:
        torch.nn.Module: The initialized model.
    """
    if model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
    elif model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
    elif model_name == 'alexnet':
        model = models.alexnet(pretrained=True)
    else:
        raise ValueError(f"Unknown model name: {model_name}")

    # Freeze pre-trained layers
    for param in model.parameters():
        param.requires_grad = False

    # Replace final fully connected layer
    num_ftrs = model.classifier[6].in_features  # Adjust index if necessary
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)

    return model

In [4]:
# Choose your desired model
model_name = 'alexnet'  # or 'vgg16', 'alexnet'
num_classes = 1  # For binary classification

# Create the model
model = create_model(model_name, num_classes)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move the model to your device (CPU or GPU)

# ... (your training and evaluation code)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 178MB/s]


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
ls

drive/  sample_data/


In [8]:
%cd /content/drive/MyDrive/CSE/

/content/drive/MyDrive/CSE


In [9]:
ls

'Biglycan breast cancer dataset'/


In [10]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for ResNet
])

# Load your dataset
dataset = datasets.ImageFolder("Biglycan breast cancer dataset", transform=transform)

# Splitting dataset into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [11]:
ls

'Biglycan breast cancer dataset'/


In [12]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=5e-5)

In [13]:
ls

'Biglycan breast cancer dataset'/


In [16]:
import torch

# Define the checkpoint file path
checkpoint_path = 'model_checkpoint_alexnet.pth'

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).type_as(outputs))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Save the checkpoint after each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss / len(train_loader)
    }, checkpoint_path)

Epoch 1, Loss: 0.6428489089012146
Epoch 2, Loss: 0.6602249410417345
Epoch 3, Loss: 0.6445890598826938
Epoch 4, Loss: 0.6231973502371047
Epoch 5, Loss: 0.6236704521709018
Epoch 6, Loss: 0.6187152597639296
Epoch 7, Loss: 0.6102357440524631
Epoch 8, Loss: 0.629979219701555
Epoch 9, Loss: 0.6040330727895101
Epoch 10, Loss: 0.6122523612446256
Epoch 11, Loss: 0.5915265878041586
Epoch 12, Loss: 0.5873329506980048
Epoch 13, Loss: 0.5864177081320021
Epoch 14, Loss: 0.5936842958132426
Epoch 15, Loss: 0.5724545452329848
Epoch 16, Loss: 0.5712286631266276
Epoch 17, Loss: 0.5831761194599999
Epoch 18, Loss: 0.5796428057882521
Epoch 19, Loss: 0.5590155588255988
Epoch 20, Loss: 0.5617029004626803
Epoch 21, Loss: 0.5731246537632413
Epoch 22, Loss: 0.5632282727294498
Epoch 23, Loss: 0.565015627278222
Epoch 24, Loss: 0.560568587647544
Epoch 25, Loss: 0.5600853496127658
Epoch 26, Loss: 0.5686259137259589
Epoch 27, Loss: 0.5579388671451144
Epoch 28, Loss: 0.5495264927546183
Epoch 29, Loss: 0.53300486670600

In [20]:
ls

'Biglycan breast cancer dataset'/   model_checkpoint_alexnet.pth


In [18]:
import torch

# Define the checkpoint file path
checkpoint_path = 'model_checkpoint_alexnet.pth'

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)
start_epoch = checkpoint['epoch'] + 1
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
loss = checkpoint['loss']

print(f"Resuming training from epoch {start_epoch}, with loss: {loss}")

num_epochs = 100
for epoch in range(start_epoch, num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).type_as(outputs))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Save the checkpoint after each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss / len(train_loader)
    }, checkpoint_path)

Resuming training from epoch 100, with loss: 0.4655163884162903


In [22]:
import torch

# Load the saved model weights
model.load_state_dict(torch.load('model_checkpoint_alexnet.pth'))

RuntimeError: Error(s) in loading state_dict for AlexNet:
	Missing key(s) in state_dict: "features.0.weight", "features.0.bias", "features.3.weight", "features.3.bias", "features.6.weight", "features.6.bias", "features.8.weight", "features.8.bias", "features.10.weight", "features.10.bias", "classifier.1.weight", "classifier.1.bias", "classifier.4.weight", "classifier.4.bias", "classifier.6.weight", "classifier.6.bias". 
	Unexpected key(s) in state_dict: "epoch", "model_state_dict", "optimizer_state_dict", "loss". 

In [23]:
import torch

# Load the checkpoint
checkpoint = torch.load('model_checkpoint_alexnet.pth')
model.load_state_dict(checkpoint['model_state_dict'])

# Optionally, you can also load the optimizer state and other saved data
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

In [24]:
model.eval()  # Set model to evaluation mode
all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

# Calculate and print the evaluation metrics
print("Confusion Matrix:")
print(confusion_matrix(all_targets, np.array(all_preds).flatten()))

print("\nClassification Report:")
print(classification_report(all_targets, np.array(all_preds).flatten(), target_names=['Healthy', 'Cancerous']))


Confusion Matrix:
[[32  6]
 [11 19]]

Classification Report:
              precision    recall  f1-score   support

     Healthy       0.74      0.84      0.79        38
   Cancerous       0.76      0.63      0.69        30

    accuracy                           0.75        68
   macro avg       0.75      0.74      0.74        68
weighted avg       0.75      0.75      0.75        68



In [25]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        # Apply sigmoid since we used BCEWithLogitsLoss
        predicted_probs = torch.sigmoid(outputs)

        # Convert probabilities to predicted class (0 or 1)
        predicted = predicted_probs > 0.5

        total += labels.size(0)
        correct += (predicted.flatten() == labels).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 75.00%
